# RCFM Model

## 1. Import modules and helper functions

In [1]:
# Modules
import matplotlib.pyplot as plt
import numpy as np
from math import sqrt
from scipy.optimize import curve_fit

# Helper functions from DataAid.py and DataImport.py
import DataAid
import DataImporter

# Numerically stable class of functions from Neros_v2.py
import Neros_v2

## 2. Load Galaxy Data

In [2]:
# Load Galaxy Data
sparcGalaxies = DataAid.GetGalaxyData("data/Sparc/Rotmod_LTG/")
littleDataGalaxies = DataAid.GetGalaxyData("data/little-data-things/data/")
lcmGalaxies = DataAid.GetGalaxyData("data/LCMFits/data/")

# Load Milky Way Model Data
xueSofueGalaxies = DataAid.GetGalaxyData("data/XueSofue/")
mcGaughMW = DataAid.GetGalaxyData("data/McGaugh/")

# Create array of Milky Way radius and vlum tuples from model data
MWXueSofue = np.array(xueSofueGalaxies['MW_lum'])
MWMcGaugh = np.array(mcGaughMW['MW_lumMcGaugh'])


## 3. Create Neros class instance

In [3]:
# Create Neros instance to perform calculations with the supplied Milky Way model as comparison
# Change Milky Way model by changing the variable in the parentheses
# i.e. neros_fns = Neros_v2.Neros(MWModelVariable)

neros_fns = Neros_v2.Neros(MWXueSofue)
MW_name = "MWXueSofue" # Change this if you change the MW model in neros_fns!
MW_rad = neros_fns.mw_rad
MW_vLum = neros_fns.mw_vLum
MW_phi = neros_fns.mw_phi

MW_vLum_interp_func = neros_fns.mw_vLum_interp

## 4. Designate outfile name, initialize variables, and designate galaxy sample

In [4]:
MW_phi

array([3.31593939e-07, 1.49405364e-06, 2.10183160e-06, ...,
       4.62346349e-06, 4.62350590e-06, 4.62355371e-06])

In [5]:
# This designates which galaxy sample to fit
galaxies = sparcGalaxies


## 5. Fit galaxies, print and save graphs

In [ ]:
"""
This is the main body of the model. 
It loops through galaxies in the designated sample,
it calculates vLCM, vNeros, and chiSquared,
it fits galaxy data using alpha and vLumFreeParam,
it prints the values of alpha and vLumFreeParam to the console,
and it saves the graphs to a file.
"""

# This is where the fitted alpha & vlum free parameter will be written for each galaxy
out_file = "imported-data/data_" + str(MW_name) + ".csv"
with open(out_file, 'w') as f:
    f.write('{0},{1},{2},{3},{4}\n'.format("Galaxy", "chi_square",  "alpha", "disk_scale", "bulge_scale"))
f.close()

total_chi_squared = []


for galaxyName in galaxies:
    f, ax = plt.subplots(1, figsize = (15, 15))
    
    # Extract out the needed galaxy components
    galaxy = np.array(galaxies[galaxyName])
    galaxy_rad = galaxy[:,0]
    galaxy_vObs = galaxy[:,1]
    galaxy_error = galaxy[:,2]
    galaxy_gas = galaxy[:,3]
    galaxy_disk = galaxy[:,4]
    galaxy_bulge = galaxy[:,5]

    # Just fit then extract the relevant pieces
    neros_fns.fit(galaxy_rad, galaxy_gas, galaxy_disk, galaxy_bulge, galaxy_vObs, galaxy_error)

    print(f"galaxyName is: {galaxyName}")
    fit_results = neros_fns.get_fit_results()
    for param in fit_results:
        print(f"{param} is: {fit_results[param]}")
    print("--------------------------")
    
    total_chi_squared.append(fit_results['chi_squared'])
    
    print("Running average chi_squared: {}".format(sum(total_chi_squared)/len(total_chi_squared)))

    
    # Write galaxyName, fitted parameters
    with open(out_file, 'a') as f:
        f.write(f"{galaxyName},{fit_results['chi_squared']},{fit_results['alpha']},{fit_results['disk_scale']},{fit_results['bulge_scale']},\n")
    f.close()

    trimmed_rad = neros_fns.get_rad()
    trimmed_vLum_updated = neros_fns.get_vLum_scaled()
    trimmed_vObs = neros_fns.get_vObs()
    trimmed_error = neros_fns.get_vObsError()
    vNeros = neros_fns.get_vNeros()
    
    # plot
    # y-axis scales to the maximum velocity value in the list galaxy_vObs_err_incl
    # or to the maximum value in the list vNeros, whichever is the bigger number
    y_max = max(max(trimmed_vObs + trimmed_error), max(vNeros))
    ax.set_ylim(bottom = 0, top = y_max + 15)

    # plot vObs and vNeros and updated vLum
    ax.plot(trimmed_rad, vNeros, label="{}_vNeros".format(galaxyName))
    ax.plot(trimmed_rad, trimmed_vObs, label="{}_vObs".format(galaxyName))
    ax.plot(trimmed_rad, trimmed_vLum_updated, label="{}_new_vLum".format(galaxyName))
    ax.plot([],[], ' ', label="$\chi^2$ = {}".format(fit_results['chi_squared']))

    # error bar in vObs
    for i in range(len(trimmed_rad)):
        ax.vlines(trimmed_rad[i], trimmed_vObs[i] - trimmed_error[i], trimmed_vObs[i] + trimmed_error[i])

    ax.legend(loc="upper right")

    graph_file_name = "graphs/" + str(galaxyName) + "_" + str(MW_name)

    plt.savefig(graph_file_name)


/opt/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:795: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


galaxyName is: NGC5371_rotmod
chi_squared is: 16.418343854473132
alpha is: 4.863424683254204e-08
disk_scale is: 0.8518964959601983
bulge_scale is: 1.0
--------------------------
Running average chi_squared: 16.418343854473132
galaxyName is: UGC07603_rotmod
chi_squared is: 19.138567011540243
alpha is: 2.515780498615376e-07
disk_scale is: 1.8045973827195996
bulge_scale is: 1.0
--------------------------
Running average chi_squared: 17.778455433006688
galaxyName is: NGC4138_rotmod
chi_squared is: 2.6236489400595113
alpha is: 6.503554467699864e-06
disk_scale is: 1.1200195061445373
bulge_scale is: 7.068599253997801e-05
--------------------------
Running average chi_squared: 12.726853268690961
galaxyName is: UGC12632_rotmod
chi_squared is: 1.8168999024493515
alpha is: 1.5864609981288102e-07
disk_scale is: 2.175797409279585
bulge_scale is: 1.0
--------------------------
Running average chi_squared: 9.999364927130559
galaxyName is: NGC5033_rotmod
chi_squared is: 123.81474246406087
alpha is: 0.

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


galaxyName is: NGC4013_rotmod
chi_squared is: 45.74394118905857
alpha is: 6.83021773718967e-08
disk_scale is: 0.00010403157494452437
bulge_scale is: 2.4516590840943517
--------------------------
Running average chi_squared: 30.313570127166173
galaxyName is: UGC07323_rotmod
chi_squared is: 1.3910924501219528
alpha is: 2.723917504700473e-07
disk_scale is: 1.1560442956592276
bulge_scale is: 1.0
--------------------------
Running average chi_squared: 28.99891205093689
galaxyName is: NGC3953_rotmod
chi_squared is: 0.4346237071660367
alpha is: 4.6282758524730606e-06
disk_scale is: 0.9191033774670625
bulge_scale is: 1.0
--------------------------
Running average chi_squared: 27.756986470772937
galaxyName is: NGC4559_rotmod
chi_squared is: 16.40926762517234
alpha is: 5.469986403544029e-09
disk_scale is: 1.1492235450558148
bulge_scale is: 1.0
--------------------------
Running average chi_squared: 27.284164852206246
galaxyName is: NGC6015_rotmod
chi_squared is: 66.50429765744659
alpha is: 1.117